# Schedule Learning Rate for More Improving
Callbacks API 활용  
1) ReduceLROnPlateau  
2) LearningRateScheduler

## Q. 모델의 성능이 더이상 개선되지 않을 때, 왜 Learning Rate을 감소시키는 전략을 쓸까?
이는 얼핏 생각해보면, 반 직관적(counter-intuitive)이지 않은가?  

Learning Rate이 무엇이었는지 다시 한번 상기해보자.  
        
    W ← W - lr * Gradient

### ReduceLROnPlateau
모델이 일정기간(patience)동안 개선되지 않으면(monitor)  
Learning rate을 일정비율(factor)로 감소시킴으로 loss를 낮춤

    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.1, # new_lr = lr * factor.
        patience=10,
        min_delta=0.0001, # threshold for measuring the new optimum, to only focus on significant changes.  
        cooldown=0, # number of epochs to wait before resuming normal operation after lr has been reduced.  
        min_lr=0
    )

In [1]:
# import tensorflow
import tensorflow as tf

In [2]:
# dataset download
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
# modeling
def create_model():
    model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10)
            ])

    return model

In [4]:
model = create_model()

model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [5]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0)
model.fit(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[reduce_lr])

Epoch 1/50
1500/1500 [==============================] - 21s 13ms/step - loss: 2.4687 - accuracy: 0.0999 - val_loss: 2.3026 - val_accuracy: 0.0909 - lr: 0.0010
Epoch 2/50
1500/1500 [==============================] - 12s 8ms/step - loss: 2.3031 - accuracy: 0.0991 - val_loss: 2.3026 - val_accuracy: 0.0909 - lr: 0.0010
Epoch 3/50
1500/1500 [==============================] - 12s 8ms/step - loss: 2.3034 - accuracy: 0.0936 - val_loss: 2.3026 - val_accuracy: 0.0562 - lr: 2.0000e-04
Epoch 4/50
1500/1500 [==============================] - 12s 8ms/step - loss: 2.3026 - accuracy: 0.0788 - val_loss: 2.3026 - val_accuracy: 0.0562 - lr: 4.0000e-05
Epoch 5/50
1500/1500 [==============================] - 12s 8ms/step - loss: 2.3031 - accuracy: 0.0790 - val_loss: 2.3026 - val_accuracy: 0.0562 - lr: 8.0000e-06
Epoch 6/50
1500/1500 [==============================] - 12s 8ms/step - loss: 2.3028 - accuracy: 0.0816 - val_loss: 2.3026 - val_accuracy: 0.0562 - lr: 1.6000e-06
Epoch 7/50
1500/1500 [=============

### LearningRateScheduler
scheduler함수 설정을 바탕으로, epoch마다 learning rate을 변환

In [6]:
def scheduler(epoch, lr):
   if epoch < 2:
     return lr
   else:
     return lr * 0.1

In [7]:
model = create_model()

model.compile(tf.keras.optimizers.Adam(lr=0.001), loss='mse')

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 512)               401920    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0)

In [9]:
model.fit(x=x_train,
          y=y_train,
          epochs=5,
          validation_data=(x_test, y_test),
          batch_size = 10000,
          callbacks=[lr_schedule])

Epoch 1/5
6/6 [==============================] - 4s 465ms/step - loss: 5716.1558 - val_loss: 2419.6265 - lr: 0.0010
Epoch 2/5
6/6 [==============================] - 2s 386ms/step - loss: 2176.3799 - val_loss: 1048.3458 - lr: 0.0010
Epoch 3/5
6/6 [==============================] - 2s 410ms/step - loss: 1571.6219 - val_loss: 840.8286 - lr: 1.0000e-04
Epoch 4/5
6/6 [==============================] - 2s 377ms/step - loss: 1391.4177 - val_loss: 806.8285 - lr: 1.0000e-05
Epoch 5/5
6/6 [==============================] - 3s 503ms/step - loss: 1369.6952 - val_loss: 802.7490 - lr: 1.0000e-06


In [10]:
# lr = 0.0001 시작
# epoch마다 2배씩 증가
# lr = 0.01을 넘게 되면, 이후부터는 epoch 마다 1/5씩 감소
def my_scheduler(epoch, lr):
    thres = 0.01
    if lr < thres:
        lr *= 2
    else:
        lr *= 0.2
    return lr

## epoch 10이하 일 때 -> 10%씩 증가
## epcoh 10 이후부터는 -> 10분의 1씩 감소


In [11]:
model = create_model()

model.compile(tf.keras.optimizers.Adam(lr=0.0001), loss='mse')

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(my_scheduler, verbose=0)


In [13]:
model.fit(x=x_train,
          y=y_train,
          epochs=5,
          validation_data=(x_test, y_test),
          batch_size = 10000,
          callbacks=[lr_schedule])

Epoch 1/5
6/6 [==============================] - 3s 408ms/step - loss: 14246.7334 - val_loss: 4887.7974 - lr: 0.0020
Epoch 2/5
6/6 [==============================] - 3s 615ms/step - loss: 2027.1410 - val_loss: 504.0517 - lr: 0.0040
Epoch 3/5
6/6 [==============================] - 3s 421ms/step - loss: 208.8319 - val_loss: 41.4968 - lr: 0.0080
Epoch 4/5
6/6 [==============================] - 2s 379ms/step - loss: 34.0943 - val_loss: 27.7147 - lr: 0.0160
Epoch 5/5
6/6 [==============================] - 2s 392ms/step - loss: 27.9414 - val_loss: 27.6063 - lr: 0.0032


## 과제
아래 조건에 맞춰 콜백을 이용하여 cifar 10 데이터를 resnet 50으로 학습을 시켜봅시다.

- early stopping : 50 에폭을 잡고, val loss 가 더이상 개선되지 않을 때 조기 종료 (세부 설정은 자율적으로 설정)
- model save : weights만 저장 (세부 설정을 자율)
- learning rate schedule : reduceLR을 사용하여, 성능개선이 되지 않을 때 10분의 1씩 감소(patience 가 early stopping patience보다 작게 잡을 것)  
학습이 완료되면, 저장된 weights를 불러와서 검증해봅시다.